## Lab01: Collect and preprocess data

- Name: Nguyễn Thế Hoàng
- Student code: 20120090

***
## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:
```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:** 

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
    - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>

# 1. Set-up environment

In [ ]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from xml.etree import ElementTree as ET
import re
# If you need other support packages

# 2. Collect data from a website by parsing HTML (4p)

In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

+ `SKU`: ID of Pokemon
+ `Name`: Name of Pokemon
+ `Price`: Price of Pokemon 
+ `InStock`: Quantity of Pokemons in stock
+ `Categories`: The category of Pokemon
+ `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).

In [ ]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

In [ ]:
def parseSKU(r):
    tags = r.html.find('.sku', first=True)

    return int(tags.text)

def parseName(r):
    tags = r.html.find('.product_title.entry-title', first=True)

    return tags.text

def parsePrice(r):
    tags = r.html.find(".summary .woocommerce-Price-amount.amount", first=True)
 
    return float(tags.text[1:])

def parseInStock(r):
    tags = r.html.find(".summary .stock", first=True)

    # Using regex get only the number of items in stock in the beggining of the string
    return int(re.findall(r'\d+', tags.text)[0])

def parseCategories(r):
    tags = r.html.find("span.posted_in a", first=False)

    categories = []
    for tag in tags:
        categories.append(tag.text)
    
    return categories

def parseTags(r):
    tags = r.html.find("span.tagged_as a", first=False)

    categories = []
    for tag in tags:
        categories.append(tag.text)
    
    return categories

def collect_data(course_urls_file):
    #load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]
    
    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    session = HTMLSession()
    cnt = 0

    for url in urls_filtered:
        
        cnt += 1
        print(cnt)
        r = session.get(url)

        SKU.append( parseSKU(r))
        Names.append( parseName(r))
        Prices.append( parsePrice(r))
        InStocks.append( parseInStock(r))
        Categories.append( parseCategories(r))
        Tags.append( parseTags(r))
        #print(SKU)
        #print(Names)
        #print(Prices)
        #print(InStocks)
        #print(Categories)
        #print(Tags)
    
    # Create a dataframe with the data. The Categories and Tags are lists, so we need to convert list of string to string
    data = pd.DataFrame({'SKU': SKU, 
                         'Name': Names, 
                         'Price': Prices, 
                         'InStock': InStocks, 
                         'Categories': Categories, 
                         'Tags': Tags})
    data['Categories'] = data['Categories'].apply(lambda x: ', '.join(x))
    data['Tags'] = data['Tags'].apply(lambda x: ', '.join(x))
    
    return data

In [ ]:
#TEST
data_pokemon = collect_data("pokemon.txt") 
assert data_pokemon.shape == (755, 6)

In [ ]:
#Save to csv file with name pokemon.csv, without using any index
data_pokemon.to_csv('pokemon.csv', index=False)

# 3. Collect data using Web API (4p)

In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:
- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:
- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
    + `country_code` and `indicator_code` are provided above.
    + For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL

In [ ]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

In [ ]:
BASE_URL = 'http://api.worldbank.org/v2/'
BASE_WB = '{http://www.worldbank.org}'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL', 
             'SP.POP.TOTL.FE.IN', 
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN', 
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

#TODO (option)
# If you need other initializations

In [ ]:
def collect_data(countryCode, per_page, start_year, end_year):
    
    data = pd.DataFrame()
    dataDict = {}

    dataDict['country'] = [countryCode] * (end_year - start_year + 1)
    dataDict['year'] = []
    for i in range(end_year, start_year - 1, -1):
        dataDict['year'].append(i)

    for indicator in INDICATORS:
        # Call API
        url = f"{BASE_URL}countries/{countryCode}/indicator/{indicator}?format=xml&date={start_year}:{end_year}"
        response = requests.get(url)
        print(url)

        # Parse XML
        tree = ET.ElementTree( ET.fromstring(response.content) )
        root = tree.getroot()
        
        yearAndAttribute = []
        year = None
        value = None
        for child in root:
            for child_2 in child:
                
                if child_2.tag == f'{BASE_WB}date':
                    year = child_2.text
                if child_2.tag == f'{BASE_WB}value':
                    value = child_2.text

            yearAndAttribute.append( (year, value) )

        dataDict[indicator] = []
        for i in yearAndAttribute:
            dataDict[indicator].append(i[1])

    data = pd.DataFrame(dataDict)
    return data
    #print(data)

In [ ]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
    return data

In [ ]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
print(data_countries.shape)
assert data_countries.shape == (161, 13)

In [ ]:
# Save to csv file with name coutries.csv
data_countries.to_csv("countries.csv", index=False, header=['Country', 'Year', 'Total Population', 'Female Population', 'Male Population', 'Birth Rate', 'Death Rate', 'Male life expectancy', 'Female life expectancy', 'School enrollment, primary', 'School enrollment, tertiary', 'Primary completion rate', 'Literacy rate'])

In [ ]:
data_countries = pd.read_csv("countries.csv")
data_countries